In [25]:
import re
import sys

print("===== WEB SECURITY SCANNER (DEFENSIVE ONLY) =====")
print("Paste your web logs or URLs below.")
print("Type 'DONE' when finished:\n")

logs = ""
while True:
    line = input()
    if line.strip().upper() == "DONE":
        break
    logs += line + "\n"


# -----------------------------
# ATTACK SIGNATURES (SAFE ONLY)
# -----------------------------

ATTACKS = {
    "SQL Injection": [
        r"UNION\s+SELECT",
        r"OR\s+1=1",
        r"'--",
        r"sleep\(\d+\)",
        r"SELECT.+FROM"
    ],

    "Cross-Site Scripting (XSS)": [
        r"<script>",
        r"alert\(",
        r"onerror=",
        r"%3Cscript%3E"
    ],

    "Remote Code Execution (RCE)": [
        r"cmd=",
        r"exec=",
        r"system\(",
        r";cat",
        r";ls",
        r"&&id",
        r"wget\s+http"
    ],

    "Directory Traversal": [
        r"\.\./\.\./",
        r"%2e%2e%2f",
        r"etc/passwd",
        r"boot.ini"
    ],

    "Command Injection": [
        r";whoami",
        r";uname",
        r";cat",
        r"\|\s*nc",
        r"\|\s*bash"
    ],

    "File Inclusion": [
        r"php://",
        r"data:text",
        r"file=",
        r"include="
    ]
}

# -----------------------------
# DETECTION ENGINE
# -----------------------------

detected = {}

for attack, rules in ATTACKS.items():
    for rule in rules:
        if re.search(rule, logs, re.IGNORECASE):
            detected.setdefault(attack, 0)
            detected[attack] += 1

# -----------------------------
# REPORTING
# -----------------------------

print("\n===============================")
print("      SECURITY SCAN REPORT")
print("===============================\n")

if not detected:
    print("No threats found. Logs appear normal.")
else:
    print("⚠️  Detected Threats:")
    for attack, count in detected.items():
        print(f"- {attack}: {count} matches")

risk_level = (
    "LOW" if not detected
    else "MEDIUM" if len(detected) <= 2
    else "HIGH"
)

print(f"\nRISK LEVEL: {risk_level}")

print("\n=== RECOMMENDATIONS ===")

if not detected:
    print("- Continue monitoring. No issues detected.")
else:
    if "SQL Injection" in detected:
        print("- Enable prepared statements / parameterized queries.")
    if "XSS" in detected:
        print("- Apply output escaping & CSP headers.")
    if "Directory Traversal" in detected:
        print("- Block '..' patterns and sanitize paths.")
    if "RCE" in detected:
        print("- Validate input and block shell metacharacters (; | &&).")
    if "Command Injection" in detected:
        print("- Disable shell execution in your application.")
    if "File Inclusion" in detected:
        print("- Disable remote file includes in PHP settings.")

print("\n=====================================")


===== WEB SECURITY SCANNER (DEFENSIVE ONLY) =====
Paste your web logs or URLs below.
Type 'DONE' when finished:



 2025-11-25 15:33:42 GET /download?file=../../../../etc/passwd src=44.51.21.18 2025-11-25 15:33:43 GET /view?file=%2e%2e%2f%2e%2e%2fconfig.php src=44.51.21.18 2025-11-25 15:33:44 GET /img?path=../windows/win.ini src=92.10.55.4
 DONE



      SECURITY SCAN REPORT

⚠️  Detected Threats:
- Directory Traversal: 3 matches
- File Inclusion: 1 matches

RISK LEVEL: MEDIUM

=== RECOMMENDATIONS ===
- Block '..' patterns and sanitize paths.
- Disable remote file includes in PHP settings.

